In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_dir = 'drive/MyDrive/505/final/'

In [ ]:
data_dir = 'drive/MyDrive/505/final/'

In [ ]:
df = pd.read_csv(data_dir + 'sample.csv', header=None)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

Num GPUs Available:  0


In [2]:
df = pd.read_csv('../dataset/llm-detect-ai-generated-text/shuffled_GPT_wiki_intro.csv', header=None)

C:\Users\17569\AppData\Local\Temp\ipykernel_11952\185645508.py:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../dataset/llm-detect-ai-generated-text/shuffled_GPT_wiki_intro.csv', header=None)


In [3]:
print(df.iloc[1:10,:])


   0                                                  1  2
1  0  Hope (born 2010) is a bottlenose dolphin who i...  1
2  1  The Invasion of Umuahia (December 20-December ...  0
3  2  Morgan Adokwei Brown is a lawyer and human rig...  1
4  3  Martine Warmann (formerly Martine George; born...  1
5  4  The Last Face is a 2016 American drama film wr...  1
6  5  Placental mammals (infraclass Placentalia ) ar...  0
7  6  The People's Secretariat of Ukraine was the hi...  1
8  7  Slavic paganism or Slavic religion describes t...  1
9  8  Lieutenant Colonel John Thomas Eustace (March ...  1


In [4]:
df= df[1:50000]

In [6]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# 文本预处理
max_words = 10000  # 仅使用最常见的10000个词
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_data.iloc[:, 1])  # 第三列是文本

X_train = tokenizer.texts_to_sequences(train_data.iloc[:, 1])
X_test = tokenizer.texts_to_sequences(test_data.iloc[:, 1])

In [7]:
max_sequence_length = max(len(seq) for seq in X_train)
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_test = pad_sequences(X_test, maxlen=max_sequence_length)

# 标签预处理
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_data.iloc[:, 2])  # 第二列是标签
y_test = label_encoder.transform(test_data.iloc[:, 2])

In [8]:
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(units=hidden_units))
model.add(Dense(1, activation='sigmoid'))

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_split=0.2)

# 评估模型
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/3
1000/1000 [==============================] - 238s 236ms/step - loss: 0.2860 - accuracy: 0.8798 - val_loss: 0.2037 - val_accuracy: 0.9239
Epoch 2/3
1000/1000 [==============================] - 228s 228ms/step - loss: 0.1301 - accuracy: 0.9535 - val_loss: 0.2174 - val_accuracy: 0.9273
Epoch 3/3
313/313 [==============================] - 21s 65ms/step - loss: 0.2044 - accuracy: 0.9312
Test Accuracy: 93.12%


In [9]:
from keras.models import save_model, load_model

# 保存模型
model.save('my_model.h5')

# 加载模型
loaded_model = load_model('my_model.h5')

# 使用加载的模型进行预测或其他操作
loss, accuracy = loaded_model.evaluate(X_test, y_test)

313/313 [==============================] - 20s 61ms/step - loss: 0.2044 - accuracy: 0.9312
